In [1]:
!pip install -U -q PyDrive

You should consider upgrading via the '/Users/haibilu/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [127]:
!pip install transformers

You should consider upgrading via the '/Users/haibilu/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [128]:
import pandas as pd
import numpy as np
import os
import chardet
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras import utils as np_utils
from keras.utils.np_utils import to_categorical

import transformers
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel

import warnings
warnings.filterwarnings("ignore")

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
fileDownloaded = drive.CreateFile({'id':'1WPo4J9wi0bIXl2I5da_UYc1UqV8q_iRw'})
fileDownloaded.GetContentFile('data_init.csv')

In [129]:
with open('data_init.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1254',
 'confidence': 0.4445340489203496,
 'language': 'Turkish'}

In [163]:
#df = pd.read_csv('data_init.csv',encoding='iso-8859-1',error_bad_lines=False, index_col = False)
df = pd.read_csv('data_init.csv',encoding='iso-8859-1',error_bad_lines=False, index_col = False).sample(n=50000)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 379459 to 432791
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ProductId                  50000 non-null  int64  
 1   Manufacturer Account Name  50000 non-null  object 
 2   Brand/Label                50000 non-null  object 
 3   IDEA''s Public ID          50000 non-null  int64  
 4   UPC Number                 45746 non-null  float64
 5   EAN Number                 10130 non-null  float64
 6   Catalog Number             50000 non-null  object 
 7   Invoice Description        50000 non-null  object 
 8   Catalog Description        33753 non-null  object 
 9   UNSPSC code                49499 non-null  float64
 10  IGCC code                  33514 non-null  float64
 11  CMD Category               49498 non-null  object 
 12  GTIN                       34007 non-null  float64
dtypes: float64(5), int64(2), object(6)
memor

In [165]:
cmd_count = df['CMD Category'].value_counts()
df['CMD_COUNT'] = df['CMD Category'].map(df['CMD Category'].value_counts())
cmd_count
threthold = 100
others_count = cmd_count[cmd_count <=threthold ].count()
others_sum = cmd_count[cmd_count <= threthold].sum()
total = cmd_count.sum()
other_pct = others_sum/total*100
print("When threthold is", threthold, ", the total number of CMD categories moved to'other' categories is", others_count)
print("The total products in 'other' category is", others_sum)
print("Percentage of products that grouped in the 'other' category is", f'{other_pct:.2f}',"%")

When threthold is 100 , the total number of CMD categories moved to'other' categories is 1406
The total products in 'other' category is 15447
Percentage of products that grouped in the 'other' category is 31.21 %


In [166]:
df.head(10)

,ProductId,Manufacturer Account Name,Brand/Label,IDEA''s Public ID,UPC Number,EAN Number,Catalog Number,Invoice Description,Catalog Description,UNSPSC code,IGCC code,CMD Category,GTIN,CMD_COUNT
379459,554808,Brady,Brady®,11120,7.540000e+11,NaN,4042-A,4042-A COOLING WATER GRN/WHT STY A,"Cooling Water Pipe Marker, 0.8125"""" H x 0.8125...",55121737.0,5872.0,Pipe identification marker,7.540000e+11,136.0
293064,414926,RAB Lighting,RAB Lighting,11274,NaN,NaN,HALV18L940-MC-K/LC,HALV 25W 4000K LED FR MET S COPP LC BLK,Architectural 399 Lumens Halv Forward Light 25...,39112102.0,4060.0,LED Wall Mounted Architectural & Specification...,NaN,372.0
212505,290770,nVent,nVent CADDY,11015,7.830000e+11,8.710000e+12,STS,"CLAMP,STRUT TO STRUT,HALFSLOT,1/2INCH HE",Strut to Strut Connector,31161708.0,2841.0,Channel nuts,7.830000e+11,7.0
494351,751317,Encore Wire,ENCORE WIRE,12260,7.390000e+11,NaN,3.45E+11,AL 250 KCMIL USE-2 BLUE 2500' RL,ALUMINUM 250 KCMIL UNDERGROUND SERVICE ENTRY (...,26121661.0,1820.0,Aluminum conductor underground service entranc...,7.390000e+11,38.0
198592,269690,Siemens,Siemens,10991,7.840000e+11,NaN,9610TH-2116-CZTA,"96PM,N5M,1024,DC,1a,60H,M,X,T,X",NaN,41113708.0,NaN,Power meters,NaN,341.0
32966,39393,Panduit,Panduit,10965,7.498382e+10,7.498382e+10,F623RSNSNSNM044,"OM1,2-Fiber,3mm,PC,Risr,SC-SC,44M","OM1 2 fiber 3mm jacket patchcord, riser (OFNR)...",43223326.0,6967.0,Fiber optic patch cord,7.498382e+10,227.0
290173,410414,Panduit,Panduit,10965,7.498390e+10,7.498390e+10,FXTRL5N5NANM008,"OM3,12-Fiber,LSZH,MPO/Fm-MPO/Fm,8M","OM3 12 fiber, interconnect cable assembly, low...",26121650.0,1808.0,Fiber optic cable assembly,7.498390e+10,620.0
338981,488199,Siemens,Siemens,10991,7.550000e+11,4.010000e+12,3RT1944-5BF42,"DC CAGE COIL,S3,110VDC",NaN,39121010.0,NaN,Magnetic coils,NaN,11.0
495051,752558,Leviton - Canada,LEVITON,12203,7.847731e+10,NaN,P1044-10L,PDU 0RU 120V 20A (24) 5-20R L5-20P/10',Vertical Power Distribution Unit 120 Volt 20 A...,39121017.0,4155.0,Power distribution units PDUs,7.847731e+10,47.0
150062,198292,Leviton - Canada,LEVITON,12203,7.847773e+10,NaN,G5362-WTE,EB IND GRAD WR TR ST GFCI 20A125V,"20 Amp, 125 Volt, SmartlockPro Self-Test GFCI ...",39121406.0,4536.0,Electrical receptacles,7.847773e+10,801.0


In [167]:
df.loc[df["CMD_COUNT"] <= 100, "CMD Category"] = "Other"

In [168]:
cmd_count = df['CMD Category'].value_counts()
cmd_count

Other                                                  15447
Cable tray fitting and accessories                      1805
Circuit breakers                                        1579
Molded case circuit breakers                            1480
Electrical equipment maintenance and repair service     1218
                                                       ...  
Electrical hangers                                       106
Area lighting                                            104
Load centers                                             104
Circular connectors                                      102
Fiber optic cable                                        101
Name: CMD Category, Length: 114, dtype: int64

In [169]:
df = df[df['CMD Category']!= 'Other']

In [170]:
X = df['Invoice Description'].astype('string')
y = df['CMD Category'].astype('str')


In [183]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state = 0)
encoder = LabelEncoder()





In [184]:
encoder.fit(y_train)

LabelEncoder()

In [185]:
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [186]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [187]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test .shape)

(24187,)
(10366,)
(24187, 114)
(10366, 114)


In [188]:
# Creating tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [189]:
bert = TFAutoModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [178]:
for layer in bert.layers:
      layer.trainable = True

In [190]:
# Tokenization of the data
def text_encode(text, tokenizer, max_len=100):
    tokens = text.apply(lambda x: tokenizer(x,return_tensors='tf', 
                                            truncation=True,
                                            padding='max_length',
                                            max_length=max_len, 
                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)

    return [input_ids,attention_mask]

In [191]:
X_train_input_ids, X_train_attention_mask = text_encode(X_train, tokenizer, max_len=100)
X_test_input_ids, X_test_attention_mask = text_encode(X_test, tokenizer, max_len=100)

In [192]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [193]:
# model creation
def build_model(bert_model, maxlen=100):
   input_ids = tf.keras.Input(shape=(maxlen,),dtype=tf.int32, name='input_ids')
   attention_mask = tf.keras.Input(shape=(maxlen,),dtype=tf.int32, name='attention_mask')

   sequence_output = bert_model(input_ids,attention_mask=attention_mask)
   output = sequence_output[0][:,0,:]
   output = tf.keras.layers.Dense(32,activation='relu')(output)
   output = tf.keras.layers.Dropout(0.2)(output)
   output = tf.keras.layers.Dense(114,activation='softmax')(output)

   model = tf.keras.models.Model(inputs = [input_ids,attention_mask], outputs = [output])
   model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

   return model

In [194]:
model_bert = build_model(bert, maxlen=100)

In [195]:
history = model_bert.fit(
    [X_train_input_ids, X_train_attention_mask],
    y_train,
    batch_size=64,
    validation_data=([X_test_input_ids, X_test_attention_mask], y_test),
    epochs=10
)

Epoch 1/10
378/378 [==============================] - 5446s 14s/step - loss: 3.8942 - accuracy: 0.2029 - val_loss: 2.9528 - val_accuracy: 0.4206
Epoch 2/10
378/378 [==============================] - 4102s 11s/step - loss: 2.7994 - accuracy: 0.4222 - val_loss: 2.2759 - val_accuracy: 0.5236
Epoch 3/10
378/378 [==============================] - 3861s 10s/step - loss: 2.2973 - accuracy: 0.5128 - val_loss: 1.9161 - val_accuracy: 0.5967
Epoch 4/10
378/378 [==============================] - 3671s 10s/step - loss: 1.9706 - accuracy: 0.5773 - val_loss: 1.6915 - val_accuracy: 0.6392
Epoch 5/10
378/378 [==============================] - 3615s 10s/step - loss: 1.7349 - accuracy: 0.6226 - val_loss: 1.5333 - val_accuracy: 0.6664
Epoch 6/10
378/378 [==============================] - 3527s 9s/step - loss: 1.5474 - accuracy: 0.6585 - val_loss: 1.4121 - val_accuracy: 0.6952
Epoch 7/10
 87/378 [=====>........................] - ETA: 47:32 - loss: 1.4062 - accuracy: 0.6827

KeyboardInterrupt: 